In [ ]:
// boring imports
var Plot = require('plotly-notebook-js');
var {loadLabelledWine, plotClustersWithLabels} = require('./utils');
var max = require('lodash/max');
var findIndex = require('lodash/findIndex');

# Neural Nets on Wine

You know the drill let's load up the Wine dataset (again) and see how a neural network performs on it with [convnetjs](http://cs.stanford.edu/people/karpathy/convnetjs/)

In [ ]:
var {features, dataset} = loadLabelledWine({verbose: true});

When using Neural Networks it's best to normalise our data

In [ ]:
var preprocess = require('ml-preprocess');

var inputs = preprocess.center(dataset.map(d => [d[0], d[10]]));
var labels = dataset.map(d => d[13]-1);

### Go ConvNet

We are going to build a simple neural network to do supervised classification. 

We're aiming for a (Fully Connected) Feed Forward Network with 1 or more hidden layers. It'll be trained using all of our sample data.

![Feed Forward Neural Network](images/feedforwardnetwork.png)




In [ ]:
var convnetjs, {Vol, Net, Layer, Trainer} = require('convnetjs')

### Load the training data into Vols

We are actually loading this into multiple copies of the input layer

In [ ]:
var X = inputs.map(input => (new Vol(input)));

### Define the Network Structure

In [ ]:
var layer_defs = [];

layer_defs.push({type:'input', out_sx:1, out_sy:1, out_depth:2});
layer_defs.push({type:'fc', num_neurons:20, activation: 'sigmoid'});
layer_defs.push({type:'softmax', num_classes:3});

// create a net
var net = new Net();
net.makeLayers(layer_defs);

### Train the network

In [ ]:
var trainer = new Trainer(net, { learning_rate:0.01, l2_decay:0.001 });

var NUM_PASSES = 10000;
for (var n = 0; n < NUM_PASSES; n++) {   
    for (var i = 0; i < 160; i++) {
        trainer.train(X[i], labels[i]);    
    }    
}

### Make Predictions


In [ ]:
var predictions = X.map((x, i) => {
    var scores = net.forward(x);
    var maxp = max(scores.w);
    return findIndex(scores.w, s => s === maxp);
});

### Plot Results

In [ ]:
$$html$$ = plotClustersWithLabels(inputs.map(d => d[0]), inputs.map(d => d[1]), labels, "Actual Labels");

In [ ]:
$$html$$ = plotClustersWithLabels(inputs.map(d => d[0]), inputs.map(d => d[1]), predictions, "Predicted Labels");

### Accuracy


In [ ]:
var ConfusionMatrix = require('ml-confusion-matrix');

var C = ConfusionMatrix.fromLabels(labels, predictions)

var M = C.getMatrix();
var trace = { 
    x: [0,1,2],
    y: [0,1,2],
    z: M,
    type: 'heatmap',
    showscale: false,
    colorscale:[[0, '#3D9970'], [100, '#001f3f']]
};

console.log("Accuracy", C.getAccuracy())
console.log("F1 Class 1", C.getF1Score(0))
console.log("F1 Class 2", C.getF1Score(1))
console.log("F1 Class 3", C.getF1Score(2))

// everything below here is just plotting code, get it in a utility fn!
var annotations = [];
M.map((a,y) => {
    a.map((b,x) => {
        annotations.push(
            {
                x: x,
                y: y,
                text: M[y][x],
                font: {
                    family: 'Arial',
                    size: 12,
                    color: 'white'
                  },
                showarrow: false
            }
        )
    })
})

var layout = { 
    xaxis: { title: "predicted", side: 'top' },
    yaxis: { title: "actuals", nticks: 6, autosize: false, autorange: 'reversed' },
    annotations,
    width: 500, height: 500};

$$html$$ = Plot.createPlot([trace], layout).render();